# Multi-Modal Weather Classification CNN

This notebook implements a Convolutional Neural Network (CNN) for weather classification using both image data and weather measurements. The system combines:

1. Image features from sky images
2. Weather measurements (cloud coverage, irradiance, sun obscuration)
3. Time-based features (hour, month)

The model architecture uses two branches:
- CNN branch for processing images
- Dense network branch for weather features
These branches are then combined for final classification.

## 1. Required Imports and Configuration

First, we'll import the required libraries and set up our configuration parameters. For Google Colab, we need to ensure all dependencies are installed.

In [1]:
# Imports
import time
import math
import random
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import json as json
import cv2

from sklearn.metrics import confusion_matrix, classification_report
from datetime import datetime, timedelta

# Add current directory to path for imports
current_dir = os.path.abspath(os.path.dirname(''))
if current_dir not in sys.path:
    sys.path.append(current_dir)

# Import our custom weather dataset with clustering
from weather_dataset import (
    read_weather_sets_with_clustering,
    get_cluster_characteristics,
    get_weather_classes_from_clusters,
    analyze_weather_distribution
 )

# Configuration and Hyperparameters
# Convolutional Layers
filter_size1 = 3 
num_filters1 = 32
filter_size2 = 3
num_filters2 = 32
filter_size3 = 3
num_filters3 = 64

# Fully-connected layers
fc_size = 128             # Number of neurons in fully-connected layer
weather_fc_size = 64      # Number of neurons for weather features

# Image configuration
num_channels = 3          # RGB images
img_size = 224           # image dimensions (square)
img_size_flat = img_size * img_size * num_channels
img_shape = (img_size, img_size)

# Weather features configuration
num_weather_features = 7  # cloud_coverage, sun_obscuration_percentage, irradiance, hour_sin, hour_cos, month_sin, month_cos
n_clusters = 5           # Fixed number of weather classes

# Training parameters
batch_size = 16
early_stopping = 10      # Number of epochs to wait for improvement

# Data paths
base_dir = r"D:\Image"  # Your image directory
test_result_dir = os.path.join(base_dir, "test_result")
json_dir = os.path.join(test_result_dir, "json")
checkpoint_dir = os.path.join(test_result_dir, "models")

# Create directories if they don't exist
os.makedirs(checkpoint_dir, exist_ok=True)

In [2]:
# Utility function: Resize and save images to disk
def resize_and_save_images(image_paths, resized_dir, size=(224, 224)):
    new_paths = []
    for img_path in image_paths:
        save_path = os.path.join(resized_dir, os.path.basename(img_path))
        if os.path.exists(save_path):
            # Skip if already resized
            new_paths.append(save_path)
            continue
        img_full_path = os.path.join(base_dir, os.path.basename(img_path))
        if not os.path.exists(img_full_path):
            print(f"Image not found: {img_full_path}")
            continue
        img = cv2.imread(img_full_path)
        if img is None:
            print(f"Could not read image: {img_full_path}")
            continue
        img_resized = cv2.resize(img, size, cv2.INTER_LINEAR)
        img_resized = cv2.cvtColor(img_resized, cv2.COLOR_BGR2RGB)
        cv2.imwrite(save_path, cv2.cvtColor(img_resized, cv2.COLOR_RGB2BGR))
        new_paths.append(save_path)
    return new_paths

In [3]:
# Load resized images from disk and match with features/labels
def load_images_from_paths(image_paths, size=(224, 224)):
    images = []
    for path in image_paths:
        img = cv2.imread(path)
        if img is None:
            print(f"Could not read image: {path}")
            continue
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, size, cv2.INTER_LINEAR)  # Just in case
        images.append(img)
    return np.array(images, dtype=np.float32) / 255.0

In [4]:
# Ablation study options: set these before running training
# Remove features by listing their names in ablation_features
# Options: 'cloud_coverage', 'sun_obscuration_percentage', 'irradiance', 'time'
# To remove images, set ablation_no_image = True
ablation_features = []  # e.g., ['cloud_coverage', 'irradiance', 'time']
ablation_no_image = False  # Set True to train without images

In [5]:
# Convert dataset to TFRecord format for efficient streaming
import tensorflow as tf
import os

def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))

def serialize_example(image, features, label):
    feature = {
        'image': _bytes_feature(image.tobytes()),
        'features': _float_feature(features.tolist()),
        'label': _float_feature(label.tolist()),
    }
    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()

def write_tfrecord(images, features, labels, filename):
    with tf.io.TFRecordWriter(filename) as writer:
        for img, feat, lbl in zip(images, features, labels):
            example = serialize_example(img, feat, lbl)
            writer.write(example)
    print(f"TFRecord written: {filename}")



In [9]:
# Read and use TFRecords in your training pipeline
import tensorflow as tf

def _parse_function(example_proto):
    feature_description = {
        'image': tf.io.FixedLenFeature([], tf.string),
        'features': tf.io.FixedLenFeature([X_train_features_resized.shape[1]], tf.float32),
        'label': tf.io.FixedLenFeature([Y_train_resized.shape[1]], tf.float32),
    }
    parsed = tf.io.parse_single_example(example_proto, feature_description)
    image = tf.io.decode_raw(parsed['image'], tf.float32)
    image = tf.reshape(image, [img_size, img_size, num_channels])
    features = parsed['features']
    label = parsed['label']
    return (image, features), label

def get_dataset_from_tfrecord(tfrecord_path, batch_size):
    dataset = tf.data.TFRecordDataset(tfrecord_path)
    dataset = dataset.map(_parse_function, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset

# Example: use train_dataset and val_dataset in model.fit
# history = model.fit(train_dataset, validation_data=val_dataset, epochs=30, callbacks=callbacks)

## 5. Training and Evaluation

1. Load and preprocess the data
2. Create and train the model
3. Evaluate the results
4. Visualize predictions

In [11]:
# Step 1: Load only image paths, features, and labels (not image arrays) to avoid memory errors
print("[Step 1] Loading weather dataset paths and features...")
data = read_weather_sets_with_clustering(
    json_dir=json_dir,
    image_dir=base_dir,
    image_size=None,  # Do not load images into memory
    use_features=True,
    n_clusters=n_clusters,
    ablation_features=ablation_features,
    ablation_no_image=ablation_no_image
 )
print("[Step 1] Done loading weather dataset paths and features.")

# Step 2: Load resized image paths from disk (already resized, do not resize again)
resized_dir = os.path.join(base_dir, 'resized224')
train_resized_paths = [os.path.join(resized_dir, os.path.basename(p)) for p in data.train.ids]
val_resized_paths = [os.path.join(resized_dir, os.path.basename(p)) for p in data.valid.ids]
test_resized_paths = [os.path.join(resized_dir, os.path.basename(p)) for p in data.test.ids]

# Features and labels remain the same order as original ids
Y_train_resized = data.train.labels
X_train_features_resized = data.train.features
Y_val_resized = data.valid.labels
X_val_features_resized = data.valid.features
Y_test_resized = data.test.labels
X_test_features_resized = data.test.features

def write_tfrecord_from_paths(image_paths, features, labels, filename, size=(224, 224)):
    with tf.io.TFRecordWriter(filename) as writer:
        for img_path, feat, lbl in zip(image_paths, features, labels):
            img = cv2.imread(img_path)
            if img is None:
                continue
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, size, cv2.INTER_LINEAR)
            example = serialize_example(img, feat, lbl)
            writer.write(example)
    print(f"TFRecord written: {filename}")

# Write train, val, test sets to TFRecord files (streaming, not loading all images)
os.makedirs('tfrecords', exist_ok=True)
write_tfrecord_from_paths(train_resized_paths, X_train_features_resized, Y_train_resized, 'tfrecords/train.tfrecord')
write_tfrecord_from_paths(val_resized_paths, X_val_features_resized, Y_val_resized, 'tfrecords/val.tfrecord')
write_tfrecord_from_paths(test_resized_paths, X_test_features_resized, Y_test_resized, 'tfrecords/test.tfrecord')

[Step 1] Loading weather dataset paths and features...
Reading weather data from JSON files...
Found 80 JSON files
Performing K-means clustering with 5 clusters...
Performing K-means clustering with 5 clusters...


c:\Users\Nitro\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Clustering completed. Cluster distribution: [36573  8657 14362  9019 25377]
Loaded 93988 images with clustered weather data
Weather class distribution: [36573  8657 14362  9019 25377]
Loaded 93988 images with clustered weather data
Weather class distribution: [36573  8657 14362  9019 25377]
[Step 1] Done loading weather dataset paths and features.
[Step 1] Done loading weather dataset paths and features.
TFRecord written: tfrecords/train.tfrecord
TFRecord written: tfrecords/train.tfrecord
TFRecord written: tfrecords/val.tfrecord
TFRecord written: tfrecords/val.tfrecord
TFRecord written: tfrecords/test.tfrecord
TFRecord written: tfrecords/test.tfrecord


In [ ]:
# TensorFlow 2.x and Keras Training (no tf.Session)
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Use tf.data datasets for training
train_dataset = get_dataset_from_tfrecord('tfrecords/train.tfrecord', batch_size)
val_dataset = get_dataset_from_tfrecord('tfrecords/val.tfrecord', batch_size)
test_dataset = get_dataset_from_tfrecord('tfrecords/test.tfrecord', batch_size)

# Image branch
image_input = layers.Input(shape=(img_size, img_size, num_channels))
x = layers.Conv2D(num_filters1, filter_size1, activation='relu', padding='same')(image_input)
x = layers.MaxPooling2D()(x)
x = layers.Conv2D(num_filters2, filter_size2, activation='relu', padding='same')(x)
x = layers.MaxPooling2D()(x)
x = layers.Conv2D(num_filters3, filter_size3, activation='relu', padding='same')(x)
x = layers.MaxPooling2D()(x)
x = layers.Flatten()(x)
x = layers.Dense(fc_size, activation='relu')(x)

# Weather branch
weather_input = layers.Input(shape=(num_weather_features,))
w = layers.Dense(weather_fc_size, activation='relu')(weather_input)

# Combine branches
combined = layers.Concatenate()([x, w])
output = layers.Dense(n_clusters, activation='softmax')(combined)

# Build and compile model
model = models.Model(inputs=[image_input, weather_input], outputs=output)
model.compile(optimizer=optimizers.Adam(1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ModelCheckpoint("best_model_all.keras", save_best_only=True)
 ]

# Train the model using tf.data datasets
print("[Step 7] Training the model with Keras (using tf.data streaming)...")
history = model.fit(train_dataset, validation_data=val_dataset, epochs=30, callbacks=callbacks)
with open('training_history_all.json', 'w') as f:
    json.dump(history.history, f)
print("Training history saved to training_history_img.json")
print("[Step 7] Model training complete.")

# Evaluate and save
val_loss, val_acc = model.evaluate(val_dataset)
print(f"Validation Accuracy: {val_acc*100:.4f}%")
model.save(os.path.join(checkpoint_dir, "weather_cnn_model_all.h5"))
model.save(os.path.join(checkpoint_dir, "weather_cnn_model_all.keras"))
print(f"Model saved to {os.path.join(checkpoint_dir, 'weather_cnn_model_all.h5')}")

In [ ]:
# EfficientNetB0: Train model using tf.data streaming
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout

# Use tf.data datasets for training
train_dataset = get_dataset_from_tfrecord('tfrecords/train.tfrecord', batch_size)
val_dataset = get_dataset_from_tfrecord('tfrecords/val.tfrecord', batch_size)
test_dataset = get_dataset_from_tfrecord('tfrecords/test.tfrecord', batch_size)

# EfficientNetB0 image branch
efficientnet_input = layers.Input(shape=(img_size, img_size, num_channels))
base_model = EfficientNetB0(include_top=False, weights='imagenet', input_tensor=efficientnet_input)
base_model.trainable = False  # Freeze base model for transfer learning
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.2)(x)
x = layers.Dense(fc_size, activation='relu')(x)

# Weather branch (same as before)
weather_input = layers.Input(shape=(num_weather_features,))
w = layers.Dense(weather_fc_size, activation='relu')(weather_input)

# Combine branches
combined = layers.Concatenate()([x, w])
output = layers.Dense(n_clusters, activation='softmax')(combined)

# Build and compile model
efficientnet_model = models.Model(inputs=[efficientnet_input, weather_input], outputs=output)
efficientnet_model.compile(optimizer=optimizers.Adam(1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

callbacks_efficientnet = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ModelCheckpoint("best_model_efficientnet.keras", save_best_only=True)
 ]

print("[Step 7B] Training the model with EfficientNetB0 (using tf.data streaming)...")
history_efficientnet = efficientnet_model.fit(train_dataset, validation_data=val_dataset, epochs=30, callbacks=callbacks_efficientnet)
with open('training_history_efficientnet.json', 'w') as f:
    json.dump(history_efficientnet.history, f)
print("Training history saved to training_history_efficientnet.json")
print("[Step 7B] EfficientNetB0 model training complete.")

# Evaluate and save
val_loss, val_acc = efficientnet_model.evaluate(val_dataset)
print(f"Validation Accuracy: {val_acc*100:.4f}%")
efficientnet_model.save(os.path.join(checkpoint_dir, "weather_cnn_model_efficientnet.h5"))
efficientnet_model.save(os.path.join(checkpoint_dir, "weather_cnn_model_efficientnet.keras"))
print(f"Model saved to {os.path.join(checkpoint_dir, 'weather_cnn_model_efficientnet.h5')}")

In [ ]:
# Train using MobileNetV3Small pretrained model (using tf.data streaming)
from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout

# Use tf.data datasets for training
train_dataset = get_dataset_from_tfrecord('tfrecords/train.tfrecord', batch_size)
val_dataset = get_dataset_from_tfrecord('tfrecords/val.tfrecord', batch_size)
test_dataset = get_dataset_from_tfrecord('tfrecords/test.tfrecord', batch_size)

# MobileNetV3Small image branch
mobilenet_input = layers.Input(shape=(img_size, img_size, num_channels))
base_model_mobilenet = MobileNetV3Small(include_top=False, weights='imagenet', input_tensor=mobilenet_input)
base_model_mobilenet.trainable = False
x = base_model_mobilenet.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.2)(x)
x = layers.Dense(fc_size, activation='relu')(x)

# Weather branch (same as before)
weather_input = layers.Input(shape=(num_weather_features,))
w = layers.Dense(weather_fc_size, activation='relu')(weather_input)

# Combine branches
combined = layers.Concatenate()([x, w])
output = layers.Dense(n_clusters, activation='softmax')(combined)

# Build and compile model
mobilenet_model = models.Model(inputs=[mobilenet_input, weather_input], outputs=output)
mobilenet_model.compile(optimizer=optimizers.Adam(1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

callbacks_mobilenet = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ModelCheckpoint("best_model_mobilenet.keras", save_best_only=True)
 ]

print("[Step 7C] Training the model with MobileNetV3Small (using tf.data streaming)...")
history_mobilenet = mobilenet_model.fit(train_dataset, validation_data=val_dataset, epochs=30, callbacks=callbacks_mobilenet)
with open('training_history_mobilenet.json', 'w') as f:
    json.dump(history_mobilenet.history, f)
print("Training history saved to training_history_mobilenet.json")
print("[Step 7C] MobileNetV3Small model training complete.")

# Evaluate and save
val_loss, val_acc = mobilenet_model.evaluate(val_dataset)
print(f"Validation Accuracy: {val_acc*100:.4f}%")
mobilenet_model.save(os.path.join(checkpoint_dir, "weather_cnn_model_mobilenet.h5"))
mobilenet_model.save(os.path.join(checkpoint_dir, "weather_cnn_model_mobilenet.keras"))
print(f"Model saved to {os.path.join(checkpoint_dir, 'weather_cnn_model_mobilenet.h5')}")

In [ ]:
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Final test evaluation
test_loss, test_acc = model.evaluate([X_test_images, X_test_features], Y_test)
print(f"Test Accuracy: {test_acc*100:.4f}%")
print(f"Test Loss: {test_loss:.4f}")

# Test set classification report
y_test_pred = model.predict([X_test_images, X_test_features])
y_test_pred_classes = np.argmax(y_test_pred, axis=1)
y_test_true_classes = np.argmax(Y_test, axis=1)

# Calculate metrics
accuracy = accuracy_score(y_test_true_classes, y_test_pred_classes)
precision = precision_score(y_test_true_classes, y_test_pred_classes, average='weighted', zero_division=0)
recall = recall_score(y_test_true_classes, y_test_pred_classes, average='weighted', zero_division=0)
f1 = f1_score(y_test_true_classes, y_test_pred_classes, average='weighted', zero_division=0)

print(f"Accuracy: {accuracy*100:.4f}%")
print(f"Precision: {precision*100:.4f}%")
print(f"Recall: {recall*100:.4f}%")
print(f"F1-score: {f1*100:.4f}%")

print(classification_report(y_test_true_classes, y_test_pred_classes, target_names=[str(c) for c in range(n_clusters)], zero_division=0))

# Plot confusion matrix for test set
cm = confusion_matrix(y_test_true_classes, y_test_pred_classes)
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=range(n_clusters), yticklabels=range(n_clusters))
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Test Set Confusion Matrix")
plt.show()

# Plot collision heatmap (misclassification matrix)
collision_matrix = np.zeros_like(cm)
for i in range(n_clusters):
    for j in range(n_clusters):
        if i != j:
            collision_matrix[i, j] = cm[i, j]
plt.figure(figsize=(8,6))
sns.heatmap(collision_matrix, annot=True, fmt='d', cmap='Reds', xticklabels=range(n_clusters), yticklabels=range(n_clusters))
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Collision Heatmap (Misclassifications)")
plt.show()

# Show example errors (misclassified images) from test set
num_examples = 9
incorrect_mask = y_test_pred_classes != y_test_true_classes
incorrect_images = X_test_images[incorrect_mask]
incorrect_true = y_test_true_classes[incorrect_mask]
incorrect_pred = y_test_pred_classes[incorrect_mask]

if len(incorrect_images) > 0:
    print("\nExample errors from test set:")
    idxs = np.random.choice(len(incorrect_images), min(num_examples, len(incorrect_images)), replace=False)
    fig, axes = plt.subplots(1, len(idxs), figsize=(15, 3))
    for i, ax in enumerate(axes):
        ax.imshow(incorrect_images[idxs[i]].reshape(img_size, img_size, num_channels))
        ax.set_title(f"True: {incorrect_true[idxs[i]]}\nPred: {incorrect_pred[idxs[i]]}")
        ax.axis('off')
    plt.show()
else:
    print("No errors found in test set!")

In [ ]:
# Load training history from JSON file
with open('training_history_efficientnet.json', 'r') as f:
    history = json.load(f)

# Plot accuracy
plt.plot(history['accuracy'], label='Train Accuracy')
plt.plot(history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Model Accuracy')
plt.show()

# Plot loss
plt.plot(history['loss'], label='Train Loss')
plt.plot(history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Model Loss')
plt.show()

In [13]:
# Load model, read test data, evaluate and visualize results
import tensorflow as tf
import numpy as np
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Choose which model to load for testing
model_path = "ablation/efficientnet/best_model_efficientnet.keras"  # Use ablation EfficientNet model
model = tf.keras.models.load_model(model_path)
print(f"Loaded model from {model_path}")

# Load test data from TFRecord
test_dataset = get_dataset_from_tfrecord('tfrecords/test.tfrecord', batch_size)

# Get predictions and true labels from test_dataset
y_true = []
y_pred = []
for (img, features), label in test_dataset:
    preds = model.predict([img, features])
    y_pred.extend(np.argmax(preds, axis=1))
    y_true.extend(np.argmax(label.numpy(), axis=1))

# Calculate metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)
recall = recall_score(y_true, y_pred, average='weighted', zero_division=0)
f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)

print(f"Accuracy: {accuracy*100:.4f}%")
print(f"Precision: {precision*100:.4f}%")
print(f"Recall: {recall*100:.4f}%")
print(f"F1-score: {f1*100:.4f}%")

print(classification_report(y_true, y_pred, target_names=[str(c) for c in range(n_clusters)], zero_division=0))

# Confusion matrix
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=range(n_clusters), yticklabels=range(n_clusters))
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Test Set Confusion Matrix")
plt.show()

# Collision heatmap (misclassification matrix)
collision_matrix = np.zeros_like(cm)
for i in range(n_clusters):
    for j in range(n_clusters):
        if i != j:
            collision_matrix[i, j] = cm[i, j]
plt.figure(figsize=(8,6))
sns.heatmap(collision_matrix, annot=True, fmt='d', cmap='Reds', xticklabels=range(n_clusters), yticklabels=range(n_clusters))
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Collision Heatmap (Misclassifications)")
plt.show()

# Show example errors (misclassified images) from test set
num_examples = 9
incorrect_mask = np.array(y_pred) != np.array(y_true)
incorrect_imgs = []
incorrect_true = []
incorrect_pred = []
for (img, features), label in test_dataset:
    preds = model.predict([img, features])
    pred_classes = np.argmax(preds, axis=1)
    true_classes = np.argmax(label.numpy(), axis=1)
    for i in range(len(img)):
        if pred_classes[i] != true_classes[i]:
            incorrect_imgs.append(img[i].numpy())
            incorrect_true.append(true_classes[i])
            incorrect_pred.append(pred_classes[i])
if len(incorrect_imgs) > 0:
    print("\nExample errors from test set:")
    idxs = np.random.choice(len(incorrect_imgs), min(num_examples, len(incorrect_imgs)), replace=False)
    fig, axes = plt.subplots(1, len(idxs), figsize=(15, 3))
    for i, ax in enumerate(axes):
        ax.imshow(incorrect_imgs[idxs[i]].reshape(img_size, img_size, num_channels))
        ax.set_title(f"True: {incorrect_true[idxs[i]]}\nPred: {incorrect_pred[idxs[i]]}")
        ax.axis('off')
    plt.show()
else:
    print("No errors found in test set!")

ValueError: Input 0 of layer "stem_conv" is incompatible with the layer: expected axis -1 of input shape to have value 3, but received input with shape (None, 225, 225, 1)